### 批量对json文件转化成png，还能进行批量resize，如512 转 1080

In [1]:
import json
import numpy as np
import cv2
# read json file
with open("./SINGLE/D5.json", "r") as f:
    data = f.read()
 
# convert str to json objs
data = json.loads(data)
 
# get the points 
points = data["shapes"][0]["points"]
points = np.array(points, dtype=np.int32)   # tips: points location must be int32
 
# read image to get shape
image = cv2.imread("./SINGLE/D5.png")
 
# create a blank image
mask = np.zeros_like(image, dtype=np.uint8)
 
# fill the contour with 255
cv2.fillPoly(mask, [points], (255, 255, 255))
 
# save the mask 
# cv2.imwrite("mask.png", mask)

True

In [3]:
# 对单个json文件生成掩膜mask
import os
def json2mask(json_path, mask_dir):
    
# read json file
    with open(json_path, "r") as f:
        data = f.read()
    # convert str to json objs
    data = json.loads(data)
    
    # get the points 
    points = data["shapes"][0]["points"]
    points = np.array(points, dtype=np.int32)   # tips: points location must be int32
    
    # # read image to get shape
    # image = cv2.imread(png_path)
    # # create a blank image
    # mask = np.zeros_like(image, dtype=np.uint8)
    mask = np.zeros((data["imageHeight"], data["imageWidth"]))
    png_name = (data["imagePath"].split("\\"))[-1].split(".")[0] + ".png"
    print(png_name)
    
    
    # fill the contour with 255
    cv2.fillPoly(mask, [points], (255, 255, 255))

    print('mask', mask.max(), mask.min())

    new_mask = np.zeros(mask.shape, dtype=np.int8)

    new_mask[mask > 0] = 1
    print('new_mask', new_mask.max(), new_mask.min())
    
    png_path = os.path.join(mask_dir, png_name)
    # save the mask 
    cv2.imwrite(png_path, new_mask)    

In [7]:
# 批量进行json2mask!!!
import json
import numpy as np
import cv2
def batch_json2mask(label_dir, mask_dir):
    os.makedirs(mask_dir, exist_ok=True)
    for root, dirs, files in os.walk(label_dir):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                print(file_path)
                name = (file_path.split("/")[-1])[:-5]
                # png_path = os.path.join(png_dir, name+".png")
                # print(png_path)
                json2mask(file_path, mask_dir)


# 指定目录
label_dir = "/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106"
mask_dir="/Users/skyous/Desktop/Li_teacher#20231105/1030/mask#20231106"

batch_json2mask(label_dir, mask_dir)
                    

/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/B6-B.json
B6-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/D5-B.json
D5-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/C4-B.json
C4-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/B8-B.json
B8-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/B10-B.json
B10-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/D3-B.json
D3-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/D10-B.json
D10-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/E3-B.json
E3-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li_teacher#20231105/1030/json#20231106/D4-B.json
D4-B.png
mask 255.0 0.0
new_mask 1 0
/Users/skyous/Desktop/Li

In [21]:
# 单独复制明场的数据集
import os
import shutil

def copy_files(source_dir, dest_dir):
    # 获取源目录下所有文件和文件夹的列表
    files = os.listdir(source_dir)
    
    for file in files:
        # 拼接源文件的完整路径
        source_file = os.path.join(source_dir, file)
        
        # 判断是否为文件以及文件名是否以"-B.png"结尾
        if os.path.isfile(source_file) and file.endswith("-B.png"):
            # 拼接目标文件的完整路径
            dest_file = os.path.join(dest_dir, file)
            
            # 复制文件
            shutil.copy2(source_file, dest_file)

# 指定源目录和目标目录
source_directory = "/Users/skyous/Desktop/train_png/LXY-80-PNG/"
destination_directory = "/Users/skyous/Desktop/train_png/imgae/"

# 调用函数进行文件复制
copy_files(source_directory, destination_directory)


In [24]:
# 手动对数据进行划分train和val，但是由于paddleseg提供有划分脚步，因此该代码仅供参考！
import os

def write_file_paths(jpeg_dir, annot_dir, output_file):

    jpeg_files = os.listdir(jpeg_dir)
    with open("./train.txt", 'w') as file:
        # 获取JPEGImages文件夹中所有文件的列表
        
        
        for jpeg_file in jpeg_files[:64]:
            # 拼接JPEG图像文件的完整路径
            
            jpeg_path = os.path.join("imgae", jpeg_file)
            
            # 获取对应的Annotations文件夹中的文件名
            annot_file = jpeg_file
            
            # 拼接Annotations文件的完整路径
            annot_path = os.path.join("mask", annot_file)
            
            # 将JPEG图像文件路径和Annotations文件路径写入txt文件
            file.write(f"{jpeg_path} {annot_path}\n")

    with open("./val.txt", 'w') as file:
        # 获取JPEGImages文件夹中所有文件的列表
        
        
        for jpeg_file in jpeg_files[64:]:
            # 拼接JPEG图像文件的完整路径
            
            jpeg_path = os.path.join("imgae", jpeg_file)
            
            # 获取对应的Annotations文件夹中的文件名
            annot_file = jpeg_file
            
            # 拼接Annotations文件的完整路径
            annot_path = os.path.join("mask", annot_file)
            
            # 将JPEG图像文件路径和Annotations文件路径写入txt文件
            file.write(f"{jpeg_path} {annot_path}\n")

# 指定JPEGImages和Annotations文件夹的路径
jpeg_images_directory = "/Users/skyous/Desktop/train_png/imgae/"
annotations_directory = "/Users/skyous/Desktop/train_png/mask/"

# 指定输出的txt文件路径
output_txt_file = "./output.txt"

# 调用函数将文件路径写入txt文件
write_file_paths(jpeg_images_directory, annotations_directory, output_txt_file)




In [13]:
# 对png进行resize成（512，512）大小
from PIL import Image
import os

def resize_png_files(input_dir, output_dir):
    # 遍历输入文件夹中的所有文件
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.endswith(".png"):
            # 拼接输入文件的完整路径
            input_path = os.path.join(input_dir, filename)
            
            # 读取图像文件
            image = Image.open(input_path)
            
            # 将图像文件reshape为(512, 512)
            resized_image = image.resize((512, 512))
            
            # 拼接输出文件的完整路径
            output_path = os.path.join(output_dir, filename)
            
            # 保存reshape后的图像文件
            resized_image.save(output_path)

# 指定输入文件夹和输出文件夹的路径
input_directory = '20231103-ZYL-ALL/1030-D0-2-single_B'
output_directory = '20231103-ZYL-ALL/1030-D0-2-single_B_512'

# 调用函数reshape PNG文件
resize_png_files(input_directory, output_directory)


In [52]:
# 扩展label为3通道，用于解决bug，但是没能解决，因此弃用！

import os
import glob
import numpy as np
from PIL import Image

# 指定PNG文件所在的文件夹路径
folder_path = "/Users/skyous/Desktop/train_png/mask_512/"

save_path = "/Users/skyous/Desktop/train_png/mask_512_3c"
os.makedirs(save_path, exist_ok=True)
# 获取文件夹中所有PNG文件的路径列表
png_files = glob.glob(os.path.join(folder_path, "*.png"))

# 遍历PNG文件列表，对每个文件进行上述操作
for png_file in png_files:
    # 读取PNG图像
    img = Image.open(png_file)

    # 将PNG图像转换为NumPy数组，并将其归一化到[0,1]范围内
    img_arr = np.array(img) / 255.

    # 创建一个全为0的（512，512，2）数组
    zeros_arr = np.zeros((512, 512, 2))

    # 将全为0数组和原始PNG图像数组沿第三个维度合并成一个（512，512，3）数组
    merged_arr = np.concatenate((zeros_arr, img_arr[..., np.newaxis]), axis=2)

    # 将合并后的数组乘以255，将其还原为[0, 255]范围内的数值
    merged_arr = merged_arr * 255

    # 将数组转换为PIL图像，并保存为新的PNG文件
    merged_img = Image.fromarray(merged_arr.astype(np.uint8))
    merged_img_file = os.path.join(save_path, png_file.split("/")[-1])
    print(merged_img_file)
    merged_img.save(merged_img_file)


/Users/skyous/Desktop/train_png/mask_512_3c/72-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/37-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/56-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/13-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/54-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/11-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/69-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/70-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/35-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/49-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/74-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/31-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/50-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/15-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/28-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/52-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/17-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/9-B.png
/Users/skyous/Desktop/train_png/mask_512_3c/76-

In [54]:
# 对label进行归一化，把255转化成1
import os
from PIL import Image

# 指定PNG文件所在的文件夹路径和新文件夹路径
folder_path = "/Users/skyous/Desktop/train_png/mask_512/"
new_folder_path = "/Users/skyous/Desktop/train_png/mask_512_max1/"

# 创建新文件夹
os.makedirs(new_folder_path, exist_ok=True)

# 获取文件夹中所有PNG文件的文件名列表
file_names = os.listdir(folder_path)

# 遍历文件名列表，对每个文件进行上述操作
for file_name in file_names:
    if file_name.endswith(".png"):
        # 构建原始文件路径和新文件路径
        original_file_path = os.path.join(folder_path, file_name)
        new_file_path = os.path.join(new_folder_path, file_name)
        print(new_file_path)

        # 打开原始PNG图像
        img = Image.open(original_file_path)

        # 将图像转换为灰度图像
        img = img.convert("L")

        # 将图像像素值最大值从255改为1
        img = img.point(lambda x: x / 255.)


        # 保存处理后的图像到新文件夹
        img.save(new_file_path, "PNG", quality=100, bits=8)


/Users/skyous/Desktop/train_png/mask_512_max1/72-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/37-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/56-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/13-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/54-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/11-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/69-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/70-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/35-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/49-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/74-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/31-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/50-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/15-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/28-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/52-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/17-B.png
/Users/skyous/Desktop/train_png/mask_512_max1/9-B.png
/Users/skyo

In [16]:
# 把tif或者png进行reshape成512，512或者1080，1080
from PIL import Image
import os

# 设置输入和输出文件夹路径
input_folder = 'predict/1103-D5-2-single_B_512_mask_largest'
output_folder = 'predict/1103-D5-2-single_B_512_mask_largest_1080'

os.makedirs(output_folder, exist_ok=True)
# 遍历文件夹中的所有图像文件
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        # 打开TIFF图像文件
        image = Image.open(os.path.join(input_folder, filename))
        print(os.path.join(input_folder, filename))

        # 缩放图像大小
        resized_image = image.resize((1080, 1080))

        # 将图像保存为PNG格式
        output_filename = os.path.splitext(filename)[0] + '.png'
        output_path = os.path.join(output_folder, output_filename)
        resized_image.save(output_path, 'PNG')


predict/1103-D5-2-single_B_512_mask_largest/B8-B.png
predict/1103-D5-2-single_B_512_mask_largest/G4-B.png
predict/1103-D5-2-single_B_512_mask_largest/G10-B.png
predict/1103-D5-2-single_B_512_mask_largest/D11-B.png
predict/1103-D5-2-single_B_512_mask_largest/D9-B.png
predict/1103-D5-2-single_B_512_mask_largest/G6-B.png
predict/1103-D5-2-single_B_512_mask_largest/C3-B.png
predict/1103-D5-2-single_B_512_mask_largest/E2-B.png
predict/1103-D5-2-single_B_512_mask_largest/E6-B.png
predict/1103-D5-2-single_B_512_mask_largest/G2-B.png
predict/1103-D5-2-single_B_512_mask_largest/C7-B.png
predict/1103-D5-2-single_B_512_mask_largest/C5-B.png
predict/1103-D5-2-single_B_512_mask_largest/F9-B.png
predict/1103-D5-2-single_B_512_mask_largest/F10-B.png
predict/1103-D5-2-single_B_512_mask_largest/E11-B.png
predict/1103-D5-2-single_B_512_mask_largest/E4-B.png
predict/1103-D5-2-single_B_512_mask_largest/C2-B.png
predict/1103-D5-2-single_B_512_mask_largest/G7-B.png
predict/1103-D5-2-single_B_512_mask_larges